# 🔐 ASEMANTIX — Proof of Concept Demo

**The First Asemantic Protocol on Raw Heterogeneous Channels**

This notebook demonstrates the core functionality of the ASEMANTIX protocol:
- Fragment construction (indistinguishable from noise)
- Local validation with bounded window
- Anti-replay protection with monotonic anchor

🌐 Website: [asemantix.tech](https://asemantix.tech)

---

## Step 1: Install the package

Click the ▶️ button to install (takes ~10 seconds)

In [ ]:
!pip install git+https://github.com/asemantix/asemantic-protocol.git -q
print("✅ Installation complete!")

## Step 2: Run the Demo

Click ▶️ to see the protocol in action!

In [ ]:
from asemantic_protocol import (
    FragmentBuilder,
    FragmentValidator,
    ReceiverState,
    ValidationResult,
    generate_seed,
    generate_domain_tag,
)

# ============================================
# 1. PROVISIONING (shared out-of-band)
# ============================================
print("="*50)
print("🔑 STEP 1: PROVISIONING")
print("="*50)

domain = generate_domain_tag()
seed = generate_seed()

print(f"Domain tag: {domain.hex()[:16]}...")
print(f"Seed K0:    {seed.hex()[:16]}...")
print("\n✅ Emitter and Receiver share these parameters out-of-band")

# ============================================
# 2. CREATE EMITTER AND RECEIVER
# ============================================
print("\n" + "="*50)
print("🏗️ STEP 2: CREATE EMITTER & RECEIVER")
print("="*50)

builder = FragmentBuilder.mode_a(domain, seed)
state = ReceiverState.mode_a(seed)
validator = FragmentValidator(domain, window_size=7)

print("Emitter:   FragmentBuilder (Mode A)")
print("Receiver:  ReceiverState + FragmentValidator")
print(f"Window:    ν = 7 (tolerates 7 lost fragments)")
print(f"Anchor t:  {state.anchor}")

# ============================================
# 3. EMIT AND VALIDATE FRAGMENT F0
# ============================================
print("\n" + "="*50)
print("📤 STEP 3: EMIT FRAGMENT F0")
print("="*50)

content = b"ALARM_LEVEL_3"
print(f"Content S: {content}")

fragment0 = builder.build(content)
print(f"\nF0 = Trunc(HMAC(K0, Encode(domain, C, K0)))")
print(f"F0: {fragment0.hex()[:24]}...{fragment0.hex()[-8:]}")
print(f"Length: {len(fragment0)*8} bits (256 bits as per claim 8)")

print("\n" + "-"*50)
print("📥 RECEIVE AND VALIDATE F0")
print("-"*50)

print(f"Anchor BEFORE: t = {state.anchor}")
result0, idx0 = validator.validate_and_commit(fragment0, state, content)
print(f"Result: {result0.value.upper()} ✅")
print(f"Matched at index j* = {idx0}")
print(f"Anchor AFTER:  t = {state.anchor} (advanced to j*+1)")

builder.advance()  # K1 = KDF(K0)

# ============================================
# 4. EMIT AND VALIDATE FRAGMENT F1
# ============================================
print("\n" + "="*50)
print("📤 STEP 4: EMIT FRAGMENT F1")
print("="*50)

fragment1 = builder.build(content)
print(f"F1 = Trunc(HMAC(K1, Encode(domain, C, K1)))")
print(f"F1: {fragment1.hex()[:24]}...{fragment1.hex()[-8:]}")
print("\n⚠️ Note: F1 ≠ F0 (different seed K1 = KDF(K0))")

print("\n" + "-"*50)
print("📥 RECEIVE AND VALIDATE F1")
print("-"*50)

print(f"Anchor BEFORE: t = {state.anchor}")
result1, idx1 = validator.validate_and_commit(fragment1, state, content)
print(f"Result: {result1.value.upper()} ✅")
print(f"Matched at index j* = {idx1}")
print(f"Anchor AFTER:  t = {state.anchor}")

builder.advance()

# ============================================
# 5. REPLAY ATTACK — F0 AGAIN
# ============================================
print("\n" + "="*50)
print("⚔️ STEP 5: REPLAY ATTACK — SEND F0 AGAIN")
print("="*50)

print(f"Attacker replays F0: {fragment0.hex()[:24]}...")
print(f"\nAnchor: t = {state.anchor}")
print(f"Window: [{state.anchor}, {state.anchor + 7}]")
print(f"F0 was valid at index 0, which is OUTSIDE the window!")

print("\n" + "-"*50)
print("📥 VALIDATION ATTEMPT")
print("-"*50)

result_replay, idx_replay = validator.validate(fragment0, state, content)
print(f"Result: {result_replay.value.upper()} ❌")
print(f"Matched index: {idx_replay}")
print(f"Anchor remains: t = {state.anchor} (no state change on reject)")

# ============================================
# SUMMARY
# ============================================
print("\n" + "="*50)
print("📊 SUMMARY")
print("="*50)
print("""
✅ F0 emitted and validated (ACCEPT)
✅ F1 emitted and validated (ACCEPT)  
✅ F0 replayed and rejected (REJECT) — Anti-replay works!
✅ Anchor monotonic: 0 → 1 → 2 (never decreases)
✅ Fragments indistinguishable from random noise
✅ No metadata transmitted (no index, no timestamp)

🔐 Patent claims demonstrated:
   • Claim 1: Windowed recomputation + strict equality
   • Claim 3: Mode A with forward secrecy (KDF)
   • Claim 7: Early stop on first match
   • Claim 9: Monotonic anchor (anti-replay)
""")
print("\n🌐 Learn more: https://asemantix.tech")

---

## 🧪 Try it yourself!

Modify the content below and run the cell to see how the fragment changes:

In [ ]:
# Change this message and run!
my_message = b"YOUR_SECRET_MESSAGE"

# Create new emitter
my_domain = generate_domain_tag()
my_seed = generate_seed()
my_builder = FragmentBuilder.mode_a(my_domain, my_seed)

# Generate fragment
my_fragment = my_builder.build(my_message)

print(f"Your message: {my_message}")
print(f"Your fragment: {my_fragment.hex()}")
print(f"\n🔐 This fragment is indistinguishable from random noise!")
print(f"   No one can tell it contains '{my_message.decode()}'")

---

## 📜 License

© 2025 ASEMANTIX. All rights reserved.

**Contact:** ASEMANTIX@proton.me

**Website:** [asemantix.tech](https://asemantix.tech)